In [204]:
from bokeh.io import show, output_file
from bokeh.plotting import figure, output_file, show
import numpy as np

In [2]:
from bokeh.models import ColumnDataSource, Plot,DatetimeTickFormatter,FixedTicker

In [3]:
Datapath_f1 = 'C:\\Users\\z0043H7B\\Tien\\Data\\dubuque\\wacker\\exploreSPM\\'

In [4]:
import sys
sys.path.insert(0, Datapath_f1)

In [27]:
import Tools10
import numpy as np
import time
import os
import pandas as pd
%load_ext autoreload
%autoreload 10

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
# test use a time series figure
Datapath = '/Users/z0043H7B/Tien/Data/dubuque/Dubuque_080819/Wacker/'

In [7]:
import os
os.chdir(Datapath)

In [57]:
spm = Tools10.fastInSPM_fldin(Datapath)

In [9]:
spm.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 16263532 entries, 2018-10-17 10:46:59 to 2019-03-14 10:25:39.500000
Data columns (total 3 columns):
events    int64
device    int64
type      float64
dtypes: float64(1), int64(2)
memory usage: 496.3 MB


In [10]:
spm.head()

,events,device,type
time,,,
2018-10-17 10:46:59,174,64,NaN
2018-10-17 10:46:59,175,1,NaN
2018-10-17 10:46:59,177,1,NaN
2018-10-17 10:46:59,177,2,NaN
2018-10-17 10:46:59,177,3,NaN


In [12]:
# get number of detectors
totaldetn,detlist=Tools10.cntDetN(spm)

In [14]:
# parse the dataframe into seperate dataframe
def sep_detact(df):
    """extract single detector ons"""
    sub = Tools10.eventParse(df,4)
    sub = Tools10.parse_more(sub,0,[82])
    
    return sub

In [15]:
all_ons = sep_detact(spm)

In [58]:
# add a new datetime column
all_ons['date']=pd.to_datetime(all_ons.index)

In [77]:
new_index = pd.DatetimeIndex(start='2018-10-17 10:45:00', freq='15 min', periods=14208, yearfirst=True)

C:\Users\z0043H7B\AppData\Local\Continuum\miniconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Creating a DatetimeIndex by passing range endpoints is deprecated.  Use `pandas.date_range` instead.
  """Entry point for launching an IPython kernel.


In [164]:
col=['det'+str(x) for x in sorted(detlist)]
newdf2= pd.DataFrame(index=new_index,columns=col)

#### Data Preparation -- creating new dataframe on the intersection activity
##### As per counts on all detectors in 15 mins period

In [165]:
from tqdm import tqdm,tqdm_notebook

for ind,row in tqdm(newdf2.iterrows()):  ### Try fix tqdm_notebook later
    start = ind
    end = ind+pd.to_timedelta('15 min')
    # get in between time 
    sub = all_ons[all_ons.date.between(start,end)]
    # get the counts for that time period
    for det in sorted(detlist):
        coln = 'det'+str(det)
        newdf2.loc[ind,coln]=sum(sub.device==det)

ImportError: IntProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [141]:
newdf['totalct']=newdf.sum(axis=1)

In [142]:
newdf.head()

,det33,det35,det37,det39,det41,det43,det45,det47,totalct
2018-10-17 10:45:00,22,8,12,9,12,9,11,9,92
2018-10-17 11:00:00,31,11,9,10,43,7,10,5,126
2018-10-17 11:15:00,45,13,10,10,15,9,13,6,121
2018-10-17 11:30:00,47,8,13,14,30,6,10,11,139
2018-10-17 11:45:00,39,11,14,13,19,6,8,8,118


In [143]:
newdf['date']=pd.to_datetime(newdf.index)

In [144]:
# generate all the miscellaneous time bin columns
newdf=Tools10.addPrdTime(newdf)

In [145]:
newdf['year']=newdf.date.dt.year

In [155]:
newdf.head()

,det33,det35,det37,det39,det41,det43,det45,det47,totalct,month,day,hour,minute,day-of-week,year,min_bin
2018-10-17 10:45:00,22,8,12,9,12,9,11,9,92,10,17,10,45,2,2018,43
2018-10-17 11:00:00,31,11,9,10,43,7,10,5,126,10,17,11,0,2,2018,44
2018-10-17 11:15:00,45,13,10,10,15,9,13,6,121,10,17,11,15,2,2018,45
2018-10-17 11:30:00,47,8,13,14,30,6,10,11,139,10,17,11,30,2,2018,46
2018-10-17 11:45:00,39,11,14,13,19,6,8,8,118,10,17,11,45,2,2018,47


In [147]:
newdf['min_bin']=np.ceil((newdf.hour*60+newdf.minute).values)/15

In [148]:
newdf=newdf.drop(['date'], axis=1)

In [149]:
newdf.min_bin=newdf.min_bin.astype(int)
newdf.head()

,det33,det35,det37,det39,det41,det43,det45,det47,totalct,month,day,hour,minute,day-of-week,year,min_bin
2018-10-17 10:45:00,22,8,12,9,12,9,11,9,92,10,17,10,45,2,2018,43
2018-10-17 11:00:00,31,11,9,10,43,7,10,5,126,10,17,11,0,2,2018,44
2018-10-17 11:15:00,45,13,10,10,15,9,13,6,121,10,17,11,15,2,2018,45
2018-10-17 11:30:00,47,8,13,14,30,6,10,11,139,10,17,11,30,2,2018,46
2018-10-17 11:45:00,39,11,14,13,19,6,8,8,118,10,17,11,45,2,2018,47


In [168]:
len(newdf)/96

148.0

In [150]:
newdf.to_csv('Wacker_15minDetAct.csv')

In [209]:
newdf= pd.read_csv('Wacker_15minDetAct.csv',encoding='utf-8',parse_dates=True,index_col=0)

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95]

In [173]:
# plot total activity per 15 mins on a full figure
    # setup figure
output_file('wacker_car_cnt_activity.html')
    # setup size 
p = figure(title='Wacker detectors count',plot_width=1400, plot_height=600)
    
    # setup tickers
mjtick, mntick= Tools10.gen_dtticker(newdf, '15D', '60T')
p.vbar(x=newdf.index, width=0.5, bottom=0, top=newdf['totalct'], color="#1d49cf") 
p.xaxis.formatter=DatetimeTickFormatter(days=["%m/%d %H:%M"],months=["%m/%d %H:%M"],hours=["%m/%d %H:%M"])#,minutes=["%m/%d %H:%M"])
    # set the grid interval filling
p.xgrid.band_fill_color = "grey"
p.xgrid.band_fill_alpha = 0.05
show(p)

In [210]:
# test get an envelope to the total count time series
import numpy as np
from scipy.signal import hilbert

In [228]:
signal = newdf['totalct'].values

In [229]:
ana_signal = hilbert(signal)
amp_env = np.abs(ana_signal)

In [238]:
testsub = amp_env[0:100]

In [237]:
amp_env[0:10]

array([118.3436487 , 126.82489427, 124.18104088, 139.00637176,
       118.01357675, 120.15874123, 108.04236045, 115.23042051,
       122.3626554 , 140.00004305])

In [239]:
output_file('wacker_envtest.html')
    # setup size 
p = figure(title='Wacker detectors count',plot_width=1400, plot_height=600)
    
    # setup tickers
p.line(list(np.linspace(0,100,100)),testsub,  line_width=0.5)#,line_color="olive")
show(p)

In [240]:
# plot total activity envolope function , along with the histogram
    # setup figure
output_file('wacker_car_cnt_env.html')
    # setup size 
p = figure(title='Wacker detectors count',plot_width=1400, plot_height=600)
    
    # setup tickers
mjtick, mntick= Tools10.gen_dtticker(newdf, '15D', '60T')
p.vbar(x=newdf.index, width=0.5, bottom=0, top=newdf['totalct'], color="#1d49cf")
p.line(newdf.index, amp_env,  line_width=0.5,color="olive")
p.xaxis.formatter=DatetimeTickFormatter(days=["%m/%d %H:%M"],months=["%m/%d %H:%M"],hours=["%m/%d %H:%M"])#,minutes=["%m/%d %H:%M"])
    # set the grid interval filling
p.xgrid.band_fill_color = "grey"
p.xgrid.band_fill_alpha = 0.05

show(p)

In [294]:
# seperate out the workdays
Wkd = newdf[newdf['day-of-week'].isin([x for x in range(0,5)])]

In [295]:
# define the group for the day
Morn = [x for x in range(15,45)]
Midd = [x for x in range(44,65)]
Aft = [x for x in range(64,96)]

In [299]:
subMorn=Wkd[Wkd['min_bin'].isin(Morn)]

In [301]:
subMorn.head()
#drop the first two
subMorn.drop(subMorn.index[[0,1]], inplace = True)

In [307]:
#find the timestamp for peak, dump that to timestamp to a new dataframe

MornT=[]
ctotlct =[]
for i in range(106):
    # loop over every 15-45 cycle and then find the max and then dump the min_bin
    sub = subMorn.totalct.iloc[(i*30):(i*30+29)]
    # find max
    tmp = list(sub.values)
    indx = tmp.index(max(tmp)) 
    ind_t = sub.index[indx]
    # append this timestamp to other df
    MornT+=[ind_t]
    ctotlct +=[max(tmp)]
    

In [308]:
Wkd_MornPk = pd.DataFrame(index=MornT,data=ctotlct,columns=['count peak value'])

In [309]:
Wkd_MornPk['date']=pd.to_datetime(Wkd_MornPk.index)
Wkd_MornPk=Tools10.addPrdTime(Wkd_MornPk)

In [310]:
Wkd_MornPk=Wkd_MornPk[['count peak value','hour','minute','day-of-week']]

In [311]:
Wkd_MornPk.head(50)

,count peak value,hour,minute,day-of-week
2018-10-18 07:00:00,191,7,0,3
2018-10-19 08:15:00,185,8,15,4
2018-10-22 06:30:00,167,6,30,0
2018-10-23 07:45:00,200,7,45,1
2018-10-24 08:15:00,174,8,15,2
2018-10-25 08:15:00,188,8,15,3
2018-10-26 08:00:00,177,8,0,4
2018-10-29 09:30:00,181,9,30,0
2018-10-30 07:00:00,173,7,0,1
2018-10-31 08:45:00,184,8,45,2


In [265]:
sub

2018-10-18 03:45:00     30
2018-10-18 04:00:00     30
2018-10-18 04:15:00     62
2018-10-18 04:30:00     81
2018-10-18 04:45:00     84
2018-10-18 05:00:00     93
2018-10-18 05:15:00     91
2018-10-18 05:30:00    138
2018-10-18 05:45:00    126
2018-10-18 06:00:00    134
2018-10-18 06:15:00    134
2018-10-18 06:30:00    157
2018-10-18 06:45:00    154
2018-10-18 07:00:00    191
2018-10-18 07:15:00    159
2018-10-18 07:30:00    133
2018-10-18 07:45:00    171
2018-10-18 08:00:00    133
2018-10-18 08:15:00    163
2018-10-18 08:30:00    143
2018-10-18 08:45:00    176
2018-10-18 09:00:00    161
2018-10-18 09:15:00    161
2018-10-18 09:30:00    148
2018-10-18 09:45:00    136
2018-10-18 10:00:00    139
2018-10-18 10:15:00    123
2018-10-18 10:30:00    128
2018-10-18 10:45:00    142
Name: totalct, dtype: int64

In [257]:
tmp = list(sub.totalct.values)
ind = tmp.index(max(tmp))

In [258]:
print(tmp)
print(tmp[ind])

[30, 30, 62, 81, 84, 93, 91, 138, 126, 134, 134, 157, 154, 191, 159, 133, 171, 133, 163, 143, 176, 161, 161, 148, 136, 139, 123, 128, 142]
191
